In [2]:
import pandas as pd
import glob
import os
import re

# Data Cleaning

In [8]:
# Step 1: Load population data
population_df = pd.read_csv(
    "complete_cleaned_copenhagen_population.csv", encoding="ISO-8859-1"
)

In [9]:
population_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474 entries, 0 to 473
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   neighborhood_id    474 non-null    int64 
 1   neighborhood_name  474 non-null    object
 2   postal_code        474 non-null    int64 
 3   postal_area        474 non-null    object
 4   postal_code_m2     474 non-null    object
 5   Total              474 non-null    int64 
 6   Men                474 non-null    int64 
 7   Women              474 non-null    int64 
dtypes: int64(5), object(3)
memory usage: 29.8+ KB


In [12]:
# Cerca tutti i file CSV con nomi coerenti
file_paths = glob.glob("DatiTure/scraped_companies_*.csv")

all_dfs = []

for path in file_paths:
    filename = os.path.basename(path)

    # Estrai il codice a 6 cifre
    try:
        code = filename.split("_")[2]
    except IndexError:
        code = "000000"  # fallback se il nome non è nel formato atteso

    # Determina se è attivo
    is_active = filename.lower().endswith("active.csv")

    # Carica il file
    df = pd.read_csv(path)

    # Aggiungi le colonne
    df["code"] = code
    df["active"] = is_active

    all_dfs.append(df)

# Unisci tutto in un unico DataFrame
merged_df = pd.concat(all_dfs, ignore_index=True)

# Salva il risultato
merged_df.to_csv("merged_companies.csv", index=False)

print("✅ Unione completata. File salvato come merged_companies.csv")

✅ Unione completata. File salvato come merged_companies.csv


In [17]:
# Funzione per estrarre il codice postale a 4 cifre dopo la prima virgola
def extract_postal_code(address):
    try:
        # Cerca la sequenza ", XXXX" dove X sono cifre
        match = re.search(r",\s*(\d{4})", address)
        return match.group(1) if match else None
    except:
        return None


# Applica la funzione alla colonna Address
merged_df["postal_code"] = merged_df["Address"].apply(extract_postal_code)

In [ ]:
# merged_df = merged_df.drop(columns=["Status"])
# merged_df = merged_df.drop(columns=["Company Type"])
merged_df.head()

,Name,Address,P-nummer,Startdate,Enddate,code,active,postal_code
0,Frankies Pizza Nørrebro,"Sortedam Dossering 3B, 2200 København N",1019849178,30.10.2014,01.05.2025,561110,True,2200
1,Søartilleriet,"Jorisvej 5, 2. 108, 2300 København S",1029736347,12.10.2023,NaN,561110,True,2300
2,Indian Flames,"Griffenfeldsgade 13, 2200 København N",1029776292,01.11.2023,NaN,561110,True,2200
3,Veksø cafe & Restaurant ApS,"Veksø Bygade 13, 3670 Veksø Sjælland",1029823061,08.11.2023,NaN,561110,True,3670
4,Khun Juk ApS,"Nyhavn 63D, 1051 København K",1029838999,15.11.2023,NaN,561110,True,1051


In [33]:
population_df = population_df.rename(columns={"neighborhood_id": "neighborhood_code"})
population_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474 entries, 0 to 473
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   neighborhood_code       474 non-null    int64  
 1   neighborhood_name       474 non-null    object 
 2   postal_code             474 non-null    object 
 3   postal_area             474 non-null    object 
 4   postal_code_m2          474 non-null    int32  
 5   Total                   474 non-null    int64  
 6   Men                     474 non-null    int64  
 7   Women                   474 non-null    int64  
 8   population_density_km2  471 non-null    float64
 9   restaurant_count        474 non-null    int32  
dtypes: float64(1), int32(2), int64(4), object(3)
memory usage: 33.5+ KB


In [29]:
population_df = population_df.replace("-", 0)

In [ ]:
# Converte postal_code in stringa per evitare problemi nei join
population_df["postal_code"] = population_df["postal_code"].astype(str)
population_df["postal_code_m2"] = population_df["postal_code_m2"].astype(int)
population_df[""] = population_df["postal_code_m2"].astype(int)
population_df["postal_code_m2"] = population_df["postal_code_m2"].astype(int)
merged_df["postal_code"] = merged_df["postal_code"].astype(str)


# 1. Densità di popolazione (abitanti per km²)
population_df["population_density_km2"] = population_df["Total"] / (
    population_df["postal_code_m2"] / 1_000_000
)

# 2. Conteggio ristoranti per codice postale
restaurant_counts = (
    merged_df.groupby("postal_code").size().reset_index(name="restaurant_count")
)

# 3. Join dei ristoranti con il DataFrame della popolazione
population_df = population_df.merge(restaurant_counts, on="postal_code", how="left")
population_df["restaurant_count"] = (
    population_df["restaurant_count"].fillna(0).astype(int)
)

# Salva il nuovo dataset
population_df.to_csv("population_with_density_and_restaurants.csv", index=False)

In [31]:
population_df.head()

,neighborhood_id,neighborhood_name,postal_code,postal_area,postal_code_m2,Total,Men,Women,population_density_km2,restaurant_count
0,101,København,1050,København K,39053,20,10,10,512.124549,16
1,101,København,1051,København K,55643,399,191,208,7170.713297,28
2,101,København,1052,København K,6563,421,204,217,64147.493524,2
3,101,København,1053,København K,2434,404,181,223,165981.922761,2
4,101,København,1054,København K,4953,392,201,191,79143.953160,1


# Sruvival Analysis

We need to prepare the data for the first part of the survival analysis.

1. We start with the Kaplan-Meier model to explore survival curves and detect patterns.
2. After that, we'll integrate the Cox PH to quantify what matters and predict survival risk using your features.

In [13]:
# Dataset preparation
#  name | start_date | end_date | is_closed | duration_days | group
restaurants_df = pd.read_csv("restaurants_df.csv")
survival_df = restaurants_df[["Name", "Startdate", "Enddate", "active", "postal_code"]]

# Convert Startdate and Enddate to datetime
survival_df["Startdate"] = pd.to_datetime(survival_df["Startdate"], errors="coerce")
survival_df["Enddate"] = pd.to_datetime(survival_df["Enddate"], errors="coerce")


# Today's date (for open restaurants)
today = pd.to_datetime("today")

# Duration in days
survival_df["end_calc"] = survival_df["Enddate"].fillna(today)
survival_df["duration_days"] = (
    survival_df["end_calc"] - survival_df["Startdate"]
).dt.days

# Event observed: 1 if restaurant closed, 0 if still active
survival_df["is_closed"] = survival_df["active"].apply(lambda x: 0 if x else 1)

survival_df.head()

C:\Users\Dell\AppData\Local\Temp\ipykernel_22064\1719573381.py:7: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  survival_df["Startdate"] = pd.to_datetime(survival_df["Startdate"], errors="coerce")
C:\Users\Dell\AppData\Local\Temp\ipykernel_22064\1719573381.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survival_df["Startdate"] = pd.to_datetime(survival_df["Startdate"], errors="coerce")
C:\Users\Dell\AppData\Local\Temp\ipykernel_22064\1719573381.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,Name,Startdate,Enddate,active,postal_code,end_calc,duration_days,is_closed
0,Frankies Pizza Nørrebro,2014-10-30,2025-01-05,True,2200,2025-01-05 00:00:00.000000,3720.0,0
1,Søartilleriet,2023-10-12,NaT,True,2300,2025-04-19 13:17:33.316578,555.0,0
2,Indian Flames,2023-11-01,NaT,True,2200,2025-04-19 13:17:33.316578,535.0,0
3,Veksø cafe & Restaurant ApS,2023-11-08,NaT,True,3670,2025-04-19 13:17:33.316578,528.0,0
4,Khun Juk ApS,2023-11-15,NaT,True,1051,2025-04-19 13:17:33.316578,521.0,0


In [14]:
!pip install lifelines

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/349.3 kB ? eta -:--:--
   -- ------------------------------------ 20.5/349.3 kB 640.0 kB/s eta 0:00:01
   ----------- ---------------------------- 102.4/349.3 kB 1.2 MB/s eta 0:00:01
   ---------------------------- ----------- 245.8/349.3 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 349.3/349.3 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/52.5 kB ? eta -:--:--
   -------------------------------------- - 51.2/52.5 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 52.5/52.5 kB 899.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/115.7 kB ? eta -:--:--
   -------------------------------------- - 112.6/115.7 kB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 115.7/115.7 kB 1.7 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=aut


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Dell\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt

# Load your data
# df = pd.read_csv("your_restaurant_data.csv")  # or use a DataFrame directly

# Initialize Kaplan-Meier estimator
kmf = KaplanMeierFitter()

# Plot for all restaurants
kmf.fit(
    durations=survival_df["duration_days"],
    event_observed=survival_df["is_closed"],
    label="All Restaurants",
)
kmf.plot_survival_function()
plt.title("Survival of Restaurants in Copenhagen")
plt.xlabel("Days since opening")
plt.ylabel("Probability of being open")
plt.show()

TypeError: NaNs were detected in the dataset. Try using pd.isnull to find the problematic values.

In [ ]:
pd.count(survival_df[survival_df["duration_days"].isna()])

,Name,Startdate,Enddate,active,postal_code,end_calc,duration_days,is_closed
1795,Gorm´s Fields Aps,NaT,NaT,True,2300,2025-04-19 13:17:33.316578,NaN,0
2376,Restaurant Hai Long v/J Jin,NaT,NaT,True,3500,2025-04-19 13:17:33.316578,NaN,0
2478,Den Lille Gule Kaffebar,NaT,NaT,True,1460,2025-04-19 13:17:33.316578,NaN,0
2479,Sachi Sushi Borgergade ApS,NaT,NaT,True,1300,2025-04-19 13:17:33.316578,NaN,0
2480,Cafe Noah Bistro ApS,NaT,NaT,True,3600,2025-04-19 13:17:33.316578,NaN,0
2490,Torvets Kiosk & Spillehal,NaT,NaT,True,3000,2025-04-19 13:17:33.316578,NaN,0
2491,Restaurationsselskabet Beirut ApS,NaT,NaT,True,2920,2025-04-19 13:17:33.316578,NaN,0
2492,Knages køkken,NaT,NaT,True,1420,2025-04-19 13:17:33.316578,NaN,0
2949,Skovgrillen Hillerød ApS,NaT,NaT,True,3400,2025-04-19 13:17:33.316578,NaN,0
3509,Urban Kebab,NaT,NaT,True,2500,2025-04-19 13:17:33.316578,NaN,0
